In [34]:
import pandas as pd
import json

# JSON 파일 읽기
with open('./reviewlog.json', 'r', encoding='utf-8') as file:
    review_data = json.load(file)

with open('./api.json', 'r', encoding='utf-8') as file:
    api_data = json.load(file)

# visit_data가 단일 객체가 아닌 리스트인 경우를 대비
if isinstance(review_data, dict):
    review_data = [review_data]

# 필요한 데이터를 추출하기 위해 딕셔너리 형태로 변환 (restaurantId를 키로 설정)
api_content = {item['id']: item for item in api_data['content']}

# 데이터프레임 생성 준비
final_data = []

for review in review_data:
    rest_id = review['restaurantId']
    
    # 해당 restaurantId에 대한 apilog 데이터 가져오기
    if rest_id in api_content:
        api_entry = api_content[rest_id]
        
        # 필요한 데이터 추출
        user_rating = api_entry.get('userRating')
        travel_time = api_entry.get('travelTime')
        rating = api_entry.get('rating')
        
        # 메뉴 최대 3개 추출
        menu_items = api_entry.get('menu', [])
        menu1 = menu_items[0] if len(menu_items) > 0 else None
        menu2 = menu_items[1] if len(menu_items) > 1 else None
        menu3 = menu_items[2] if len(menu_items) > 2 else None
        
        # 카테고리 최대 2개 id 추출
        category_ids = [cat['id'] for cat in api_entry.get('categories', [])]
        category_id1 = category_ids[0] if len(category_ids) > 0 else None
        category_id2 = category_ids[1] if len(category_ids) > 1 else None
        
        # visit 정보와 결합
        final_data.append({
            '유저 아이디': review['author']['id'],
            '방문한 레스토랑 아이디': rest_id,
            '방문일시': review['createdAt'],
            'reviewRating': review['rating'],
            'reviewText': review['content'],
            'userRating': user_rating,
            'travelTime': travel_time,
            'rating': rating,
            'store_menu1': menu1,
            'store_menu2': menu2,
            'store_menu3': menu3,
            'food_category_id1': category_id1,
            'food_category_id2': category_id2
        })

# 데이터프레임 생성
df = pd.DataFrame(final_data)

# 결과 출력
df


,유저 아이디,방문한 레스토랑 아이디,방문일시,reviewRating,reviewText,userRating,travelTime,rating,store_menu1,store_menu2,store_menu3,food_category_id1,food_category_id2
0,2001,19,2024-10-27T15:50:00,4.5,The food was great and the service was excellent.,5.0,1,4.5,커피,None,None,7,None
1,2002,19,2024-10-27T15:53:00,3.5,The food was great and the service was excellent.,5.0,1,4.5,커피,None,None,7,None


In [36]:
processed_df = df.groupby(['유저 아이디','방문한 레스토랑 아이디']).apply(lambda group: pd.Series({
    '리뷰일시': group['방문일시'].iloc[0],
    '처음해당 식당리뷰일시': group['방문일시'].min(),
    '최근해당 식당리뷰일시': group['방문일시'].max(),
    '해당유저 해당식당 리뷰개수': group.shape[0],
    'reviewRating': group['userRating'].iloc[0],
    'reviewText': group['reviewText'].iloc[0],
    'userRating': group['userRating'].iloc[0],
    'travelTime': group['travelTime'].iloc[0],
    'rating': group['rating'].iloc[0],
    'menu1': group['store_menu1'].iloc[0],
    'menu2': group['store_menu2'].iloc[0],
    'menu3': group['store_menu3'].iloc[0],
    'food_category_id1': group['food_category_id1'].iloc[0],
    'food_category_id2': group['food_category_id2'].iloc[0],
})).reset_index()

# 결과 출력
processed_df

,유저 아이디,방문한 레스토랑 아이디,리뷰일시,처음해당 식당리뷰일시,최근해당 식당리뷰일시,해당유저 해당식당 리뷰개수,reviewRating,reviewText,userRating,travelTime,rating,menu1,menu2,menu3,food_category_id1,food_category_id2
0,2001,19,2024-10-27T15:50:00,2024-10-27T15:50:00,2024-10-27T15:50:00,1,5.0,The food was great and the service was excellent.,5.0,1,4.5,커피,None,None,7,None
1,2002,19,2024-10-27T15:53:00,2024-10-27T15:53:00,2024-10-27T15:53:00,1,5.0,The food was great and the service was excellent.,5.0,1,4.5,커피,None,None,7,None


In [38]:
processed_df['해당유저 리뷰개수'] = df.groupby(['유저 아이디']).size().values
processed_df['해당유저 해당식당 평균 리뷰평점'] = df.groupby(['유저 아이디', '방문한 레스토랑 아이디'])['userRating'].mean().values
processed_df['해당유저 평균 리뷰평점'] = df.groupby(['유저 아이디'])['userRating'].mean().values
# 결과 출력
processed_df

,유저 아이디,방문한 레스토랑 아이디,리뷰일시,처음해당 식당리뷰일시,최근해당 식당리뷰일시,해당유저 해당식당 리뷰개수,reviewRating,reviewText,userRating,travelTime,rating,menu1,menu2,menu3,food_category_id1,food_category_id2,해당유저 리뷰개수,해당유저 해당식당 평균 리뷰평점,해당유저 평균 리뷰평점
0,2001,19,2024-10-27T15:50:00,2024-10-27T15:50:00,2024-10-27T15:50:00,1,5.0,The food was great and the service was excellent.,5.0,1,4.5,커피,None,None,7,None,1,5.0,5.0
1,2002,19,2024-10-27T15:53:00,2024-10-27T15:53:00,2024-10-27T15:53:00,1,5.0,The food was great and the service was excellent.,5.0,1,4.5,커피,None,None,7,None,1,5.0,5.0


In [ ]:
from openai import OpenAI
import pandas as pd
import json
from tqdm import tqdm
from dotenv import load_dotenv
import os

# .env 파일에서 API 키 불러오기
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # 최신 방식으로 클라이언트 생성

# 최신 OpenAI API 방식을 사용한 get_completion 함수 정의
def get_completion(prompt, model="gpt-3.5-turbo-instruct"):
    completion = client.completions.create(
        model=model,
        prompt=prompt,
        temperature=0,
        max_tokens=10  # 간결한 응답을 위해 토큰 제한 설정
    )
    return completion.choices[0].text.strip()


# 긍정적 점수를 예측하는 함수 정의
def predict_sentiment_score(review_text):
    prompt = f"너가 해야할 일은 '{review_text}'를 읽고 이 리뷰가 얼마나 긍정적인지 부정적인지를 1부터 10까지 평가하는 일이고 결과물로 너는 아무 부가설명 필요 없이 점수에 해당하는 숫자 하나만 출력해"
    score_text = get_completion(prompt)
    
    try:
        score = int(score_text)  # 응답에서 숫자를 추출
    except ValueError:
        score = None  # 점수 예측에 실패한 경우 None으로 설정
    return score

# 모든 리뷰 내용에 대해 긍정적 점수를 예측하여 '리뷰점수' 컬럼에 추가
df['리뷰점수'] = [predict_sentiment_score(text) for text in tqdm(df['리뷰 내용'], desc="Predicting sentiment scores")]

# 결과 출력
df

In [7]:
import pandas as pd
import json
from datetime import datetime, timedelta  # datetime 모듈 임포트
with open('./reviewlog.json', 'r', encoding='utf-8') as file:
    review_data = json.load(file)


# 방문일시를 datetime 형식으로 변환
df['방문일시'] = pd.to_datetime(df['방문일시'])

df['방문여부'] = 0
df['reviewrating'] = None

# reviewlog 조건을 충족하는지 확인하는 함수 정의
def check_visit_and_rating(row):
    for review in review_data:
        author_id = review['author']['id']
        restaurant_id = review['restaurantId']
        created_at = datetime.strptime(review['createdAt'], "%Y-%m-%dT%H:%M:%S")
        
        # 조건에 맞는지 확인
        if (
            row['유저 아이디'] == author_id and
            row['방문한 레스토랑 아이디'] == restaurant_id and
            abs((row['방문일시'] - created_at).days) <= 1
        ):
            # 조건을 만족할 때 방문여부를 1로 하고, reviewrating 컬럼에 rating 값을 저장
            row['reviewrating'] = review.get('rating')
            row['방문여부'] = 1
            return row

    return row

# 모든 행에 대해 check_visit_and_rating 함수를 적용하여 방문여부와 reviewrating 컬럼을 생성
df = df.apply(check_visit_and_rating, axis=1)


# 결과 출력
df

,유저 아이디,방문한 레스토랑 아이디,방문일시,userRating,travelTime,rating,menu1,menu2,menu3,category_id1,category_id2,방문여부,reviewrating
0,2001,19,2024-10-27 15:45:00,5.0,1,4.5,커피,None,None,7,None,1,4.5


In [8]:
processed_df = df.groupby(['유저 아이디', '방문한 레스토랑 아이디']).apply(lambda group: pd.Series({
    '방문일시': group['방문일시'].max(),
    'userRating': group['userRating'].mean(),
    'travelTime': group['travelTime'].iloc[0],
    'rating': group['rating'].iloc[0],
    'menu1': group['menu1'].iloc[0],
    'menu2': group['menu2'].iloc[0],
    'menu3': group['menu3'].iloc[0],
    'category_id1': group['category_id1'].iloc[0],
    'category_id2': group['category_id2'].iloc[0],
    '평균 reviewRating': group['reviewrating'].mean(),
    '가장 최근 reviewRating': group.sort_values('방문일시', ascending=False)['reviewrating'].iloc[0],
    '방문여부': group['방문여부'].iloc[0]
})).reset_index()

# 결과 출력
processed_df

C:\Users\LG\AppData\Local\Temp\ipykernel_31420\1661343269.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_df = df.groupby(['유저 아이디', '방문한 레스토랑 아이디']).apply(lambda group: pd.Series({


,유저 아이디,방문한 레스토랑 아이디,방문일시,userRating,travelTime,rating,menu1,menu2,menu3,category_id1,category_id2,평균 reviewRating,가장 최근 reviewRating,방문여부
0,2001,19,2024-10-27 15:45:00,5.0,1,4.5,커피,None,None,7,None,4.5,4.5,1


In [9]:
processed_df['만족여부']=None#자연어처리할거임
processed_df

,유저 아이디,방문한 레스토랑 아이디,방문일시,userRating,travelTime,rating,menu1,menu2,menu3,category_id1,category_id2,평균 reviewRating,가장 최근 reviewRating,방문여부,만족여부
0,2001,19,2024-10-27 15:45:00,5.0,1,4.5,커피,None,None,7,None,4.5,4.5,1,None


In [28]:
import pandas as pd
import json

# JSON 파일 읽기
with open('./reviewlog.json', 'r', encoding='utf-8') as file:
    review_data = json.load(file)

# 데이터프레임 생성
# 중첩된 구조에서 필요한 필드를 분리하여 DataFrame으로 만듦
df = pd.DataFrame([{
    '리뷰 ID': review['id'],
    '작성일시': review['createdAt'],
    '평점': review['rating'],
    '리뷰 내용': review['content'],
    '레스토랑 ID': review['restaurantId'],
    '작성자 ID': review['author']['id'],
    '작성자 이름': review['author']['username'],
    '작성자 이메일': review['author']['email']
} for review in review_data])

# 작성일시를 datetime 형식으로 변환
df['작성일시'] = pd.to_datetime(df['작성일시'])

df


,리뷰 ID,작성일시,평점,리뷰 내용,레스토랑 ID,작성자 ID,작성자 이름,작성자 이메일
0,1,2024-10-27 15:30:00,4.5,The food was great and the service was excellent.,1001,2001,user123,user123@example.com
1,2,2024-10-27 15:50:00,4.5,The food was great and the service was excellent.,19,2001,user123,user123@example.com


In [27]:
from openai import OpenAI
import pandas as pd
import json
from tqdm import tqdm
from dotenv import load_dotenv
import os

# .env 파일에서 API 키 불러오기
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # 최신 방식으로 클라이언트 생성

# 최신 OpenAI API 방식을 사용한 get_completion 함수 정의
def get_completion(prompt, model="gpt-3.5-turbo-instruct"):
    completion = client.completions.create(
        model=model,
        prompt=prompt,
        temperature=0,
        max_tokens=10  # 간결한 응답을 위해 토큰 제한 설정
    )
    return completion.choices[0].text.strip()


# 긍정적 점수를 예측하는 함수 정의
def predict_sentiment_score(review_text):
    prompt = f"너가 해야할 일은 '{review_text}'를 읽고 이 리뷰가 얼마나 긍정적인지 부정적인지를 1부터 10까지 평가하는 일이고 결과물로 너는 아무 부가설명 필요 없이 점수에 해당하는 숫자 하나만 출력해"
    score_text = get_completion(prompt)
    
    try:
        score = int(score_text)  # 응답에서 숫자를 추출
    except ValueError:
        score = None  # 점수 예측에 실패한 경우 None으로 설정
    return score

# 모든 리뷰 내용에 대해 긍정적 점수를 예측하여 '리뷰점수' 컬럼에 추가
df['리뷰점수'] = [predict_sentiment_score(text) for text in tqdm(df['리뷰 내용'], desc="Predicting sentiment scores")]

# 결과 출력
df

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}